In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
 

In [3]:
# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords', download_dir='/usr/local/share/nltk_data')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Matheus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
df_train = pd.read_csv("twitter_training.csv")
df_validation = pd.read_csv("twitter_validation.csv")

df_train.columns = ['ID', 'Category', 'Sentiment', 'Tweet']
df_validation.columns = ['ID', 'Category', 'Sentiment', 'Tweet']

In [5]:
print(df_train.columns)
print(df_validation.columns)

Index(['ID', 'Category', 'Sentiment', 'Tweet'], dtype='object')
Index(['ID', 'Category', 'Sentiment', 'Tweet'], dtype='object')


In [ ]:
#Remover duplicatas
df_train.dropna(subset=['Tweet'], inplace=True)
df_validation.dropna(subset=['Tweet'], inplace=True)

In [ ]:
# 🔹 Remover tweets curtos (menos de 3 palavras, mas alterei para > 1)
df_train = df_train[df_train['Tweet'].str.split().str.len() > 1]
df_validation = df_validation[df_validation['Tweet'].str.split().str.len() > 1]


In [65]:
df_train = df_train[df_train['Sentiment'] != 'neutral']
df_validation = df_validation[df_validation['Sentiment'] != 'neutral']


In [66]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)  # Tokenização
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)

In [67]:
# Aplicar pré-processamento
df_train['Cleaned_Tweet'] = df_train['Tweet'].apply(preprocess_text)
df_validation['Cleaned_Tweet'] = df_validation['Tweet'].apply(preprocess_text)

In [68]:
# Codificar os rótulos
y = LabelEncoder().fit_transform(df_train['Sentiment'])

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(df_train['Cleaned_Tweet'], y, test_size=0.2, random_state=42)

In [ ]:
# Modelos e hiperparâmetros
vectorizers = {
    'bow': CountVectorizer(max_features=3000, ngram_range=(1, 2)), #alterei manualmmente (Notei que quanto mais features, menor a %.)
    'tfidf': TfidfVectorizer(max_features=3000, ngram_range=(1, 2)) 
}

models = {
    'logistic_regression': LogisticRegression(penalty='l2', max_iter=2000),
    'naive_bayes': GaussianNB(),
    'knn': KNeighborsClassifier(),
    'mlp': MLPClassifier(max_iter=250) #alterei manualmmente (oscilou em 0.45 a 0.5)
}

grid_params = {
    'logistic_regression': {'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]},
    'knn': {'n_neighbors': [2, 4, 8, 12]},
    'mlp': {'hidden_layer_sizes': [
            (2,), (4,), (8,), (16,), (32,),  # 1 camada
            (2, 2), (4, 4), (8, 8), (16, 16), (32, 32),  # 2 camadas
            (2, 2, 2), (4, 4, 4), (8, 8, 8), (16, 16, 16), (32, 32, 32)  # 3 camadas
        ]
    }
}


results = []

In [110]:
for vec_name, vectorizer in vectorizers.items():
    for model_name, model in models.items():
        print(f'Treinando {model_name} com {vec_name}...')
        
        pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('scaler', StandardScaler(with_mean=False)),
            ('pca', PCA(n_components=100)),  # 0.9 n funcionou, alterei manualmente. 
            ('clf', model)
        ])
        
        if model_name in grid_params:
            adjusted_params = {f'clf__{key}': value for key, value in grid_params[model_name].items()}
            grid = RandomizedSearchCV(pipeline, adjusted_params, scoring='accuracy', cv=3, n_jobs=-1)
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
        else:
            best_model = pipeline.fit(X_train, y_train)
        
        y_pred = best_model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted')
        rec = recall_score(y_test, y_pred, average='weighted')
        
        results.append((vec_name, model_name, acc, prec, rec))

Treinando logistic_regression com bow...


c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Treinando naive_bayes com bow...
Treinando knn com bow...


c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Treinando mlp com bow...


c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Treinando logistic_regression com tfidf...
Treinando naive_bayes com tfidf...
Treinando knn com tfidf...


c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Treinando mlp com tfidf...


c:\Users\Matheus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


In [111]:
# Exibir resultados
for res in results:
    print(f'Representação: {res[0]}, Modelo: {res[1]}, Acurácia: {res[2]:.4f}, Precision: {res[3]:.4f}, Recall: {res[4]:.4f}')



Representação: bow, Modelo: logistic_regression, Acurácia: 0.4745, Precision: 0.4771, Recall: 0.4745
Representação: bow, Modelo: naive_bayes, Acurácia: 0.2905, Precision: 0.4061, Recall: 0.2905
Representação: bow, Modelo: knn, Acurácia: 0.5937, Precision: 0.6166, Recall: 0.5937
Representação: bow, Modelo: mlp, Acurácia: 0.5227, Precision: 0.5230, Recall: 0.5227
Representação: tfidf, Modelo: logistic_regression, Acurácia: 0.4869, Precision: 0.4882, Recall: 0.4869
Representação: tfidf, Modelo: naive_bayes, Acurácia: 0.3107, Precision: 0.4042, Recall: 0.3107
Representação: tfidf, Modelo: knn, Acurácia: 0.5784, Precision: 0.5994, Recall: 0.5784
Representação: tfidf, Modelo: mlp, Acurácia: 0.5276, Precision: 0.5301, Recall: 0.5276
